### Brute Force approach
First we create a brute force approach. This will only work with very small n, but we can use all other approaches on equally small models and see if they create the same result

In [120]:
import numpy as np

In [121]:
N = 3
B = 1

In [122]:
def generate_potential(B):
    potentials = np.ones((2,2))
    potentials[0,0] = np.exp(B)
    potentials[1,1] = potentials[0, 0]
    return potentials

In [123]:
def generate_all_lattice_states(n):
    """given n, create a (2^(n*n))xnxn ndarray of all states

    Args:
        n (int): size of the nxn array to calculate all states
    
    return:
        states ((2^(n*n))xnxn): binary array of all states of the nxn array
    """
    # first create a flattened nxn array for each state. This makes it easier to create each state. Then we convert to 2D before returning
    states_lattice = np.zeros((pow(2, n*n), n*n)).astype(int)
    
    for state_index in range(pow(2, n*n)):
        # binary representation of the current state
        state_binary = bin(state_index)[2:]
        
        # iterate through the bit representation of the state
        for bit_index, bit_value in enumerate(reversed(state_binary)):
            states_lattice[state_index, bit_index] = bit_value
    
    return states_lattice.reshape((-1, n, n))
                

In [124]:
def generate_lattice_state_from_binary(state_index, n):
    """given n, create a nxn ndarray of state number "state"

    Args:
        state (int): generate the state "state" lattice
        n (int): size of the nxn array to calculate all states
    
    return:
        state_lattice (nxn ndarray): binary array of nxn state array
    """
    # first create a flattened nxn array for each state. This makes it easier to create each state. Then we convert to 2D before returning
    state_lattice = np.zeros(n*n).astype(int)
    
    # binary representation of the current state
    state_binary = bin(state_index)[2:]
    
    # iterate through the bit representation of the state
    for bit_index, bit_value in enumerate(reversed(state_binary)):
        state_lattice[bit_index] = bit_value
    
    return state_lattice.reshape((1, n, n))

In [125]:
def calculate_states_potential(states_lattice, potential):
    """given a (states)xnxn ndarray of binary states, return the potential of this state

    Args:
        states_lattice ([states, n,n] binary ndarray): binary nxn array
        potential ([2,2] ndarray)
    
    Return:
        state_potential : potential of the state given
    """
    n = states_lattice.shape[1]
    state_potential = np.ones(states_lattice.shape[0])
    
    for state in range(states_lattice.shape[0]):
        for row in range(n):
            for col in range(n-1):
                # Calculate potentials for all neighbouring nodes on same row (left-right)
                val_1 = states_lattice[state, row, col]
                val_2 = states_lattice[state, row, col+1]
                state_potential *= potential[val_1, val_2]
                
                # calculate potentials for all neighbouring nodes just below selected row in selected column (up-down)
                if row != n-1:
                    val_1 = states_lattice[state, row, col]
                    val_2 = states_lattice[state, row+1, col]
                    state_potential *= potential[val_1, val_2]
            
            # col value doesn't hit the final column in lattice so calculate separately:
            if row != n-1:
                val_1 = states_lattice[state, row, n-1]
                val_2 = states_lattice[state, row+1, n-1]
                state_potential *= potential[val_1, val_2]
    
    return state_potential.sum()

In [142]:
# define parameters
n = 5
B = 1
Z = 0
pot = generate_potential(B)

for state_index in range(pow(2, n*n)):
    state_lattice = generate_lattice_state_from_binary(state_index, n)
    z = calculate_states_potential(state_lattice, pot)
    Z += z

Z = Z / pow(2,n*n)

print(Z)



159957216028.8102


In [143]:
np.log(Z)

25.798172216602595

In [127]:
def f_0(y1, potential):
    """given y of dim n, return the potential of y

    Args:
        y1 (n-array): n dimensional array (this is the first column in the nxn array)
        potential : 2x2 array of potential
    
    Return:
        message: return the message from this factor
    """
    
    factor = 1
    for row in range(y1.size - 1):
        factor *= potential[y1[row], y1[row+1]]
    
    return factor

def f_0_vectorize(column_states, potential):
    """given an array of all possible column states, return an array of all potentials

    Args:
        column_states (_type_): _description_
        potential (_type_): _description_
    """
    m = column_states.shape[0]
    f0 = np.zeros(m)
    
    for y_i in range(m):
        f0[y_i] = f_0(column_states[y_i], potential)
    
    return f0

In [132]:
def f_i(y1, y2, potential):
    """given column 1 and 2 vectors, calculate the shared potential

    Args:
        y1 (n-array): column y1, binary array
        y2 (n-array): column y2, binary array
        potential (2x2 array): potential to be applied to all neighbours
    
    Return:
        message: return the message from y1 and y2
    """
    
    factor = 1
    
    # calculate potentials going down the 2nd column and shared edges between y1 and y2
    for row in range(y1.size-1):
        factor *= potential[y2[row], y2[row+1]]
        factor *= potential[y1[row], y2[row]]
    # for loop misses out last row - calculate that outside loop
    factor *= potential[y1[-1], y2[-1]]
    
    return factor

def f_i_vectorize(column_states, potential):
    """given an array of all possible column states, return an array of all potentials

    Args:
        column_states (_type_): _description_
        potential (_type_): _description_
    """
    m = column_states.shape[0]
    fi = np.zeros((m,m))
    
    for y_i in range(m):
        for y_j in range(m):
            fi[y_i, y_j] = f_i(column_states[y_i], column_states[y_j], potential)
    
    return fi

In [129]:
def generate_column_states(n):
    """given the dimension n, return an array of all possible states for binary vector n

    Args:
        n (int): dimension of vector
        
    Return:
        y : binary array of shape 2^n x n
    """
    
    y = np.zeros((pow(2, n), n)).astype(int)
    
    for state_index in range(pow(2, n)):
        state_binary = bin(state_index)[2:]
    
        for bit_index, bit_value in enumerate(reversed(state_binary)):
            y[state_index, bit_index] = bit_value
    
    return y

In [146]:
n = 3
B = 1
column_states = generate_column_states(n)
potential = generate_potential(B)

m_y1 = 0
m_y1_y2 = 0
m_y2_y3 = 0

for y3 in column_states:
    for y2 in column_states:
        for y1 in column_states:
            m_y1 = f_0(y1, potential)
            m_y1_y2 += f_i(y1, y2, potential) * m_y1
        m_y2_y3 += f_i(y2, y3, potential) * m_y1_y2
        # reset the variables
        m_y1_y2 = 0

Z = m_y2_y3 / pow(2, (n*n))

print(Z)

2107.3781632042655


### Fast way to calculate Z
Above are 2 different ways to calculate Z, one being just brute force. And they all agree upto 5 (above that it takes too long to calculate)

In [145]:
n = 3
B = 1
column_states = generate_column_states(n)
potential = generate_potential(B)

f0 = f_0_vectorize(column_states, potential)
fi = f_i_vectorize(column_states, potential)

message = f0

for col in range(1, n):
    message = message @ fi

Z = message.sum() / pow(2, (n**2))

print(Z)
print(np.log(Z))

2107.3781632042655
7.653199877151158
